In [5]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from Util import evaluate_curret, get_invest_period
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os
import time
import sklearn.covariance
import pickle
from collections import namedtuple, defaultdict
from subprocess import Popen
from multiprocessing import Pool

In [2]:
util_loaded_data = Util.load_data()
ret, ret_nonan, univ, tradeidx, dates = util_loaded_data

In [2]:
from estimators import (MTP2_wrapper, MTP2_cov_wrapper,
                        LRPS_wrapper,
                        CLIME_wrapper, CLIME_cov_wrapper,
                        old_LS_wrapper, LS_wrapper, NLS_wrapper,
                        POET_wrapper,
                        glasso_wrapper,
                        get_AFM_estimator,
                        tiger_wrapper)

In [45]:
T = 200
N = 100
with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)

In [33]:
pastRets[0].shape

(200, 100)

In [52]:
X = pastRets[0]

In [53]:
clime = CLIME_wrapper(X)

d0b86a79-aa70-4c9d-97bc-bf13f5cceab1
b'[1] "Loaded X"\n'
b'[1] 0.1\n'
b'                [,1]          [,2]          [,3]          [,4]          [,5]\n'
b'  [1,]  3.967146e+03  4.976797e-08 -1.714802e-06 -1.238584e-06 -3.021648e-07\n'
b'  [2,]  4.976797e-08  5.887499e+03 -2.010097e-06 -1.692501e-06 -8.893520e-07\n'
b'  [3,] -1.714802e-06 -2.010097e-06  1.116330e+04 -1.154531e-06 -1.234797e-06\n'
b'  [4,] -1.238584e-06 -1.692501e-06 -1.154531e-06  7.239594e+03 -3.050512e-06\n'
b'  [5,] -3.021648e-07 -8.893520e-07 -1.234797e-06 -3.050512e-06  7.966426e+03\n'
b'  [6,] -6.243866e-07 -8.903214e-07 -1.853748e-06 -8.415533e+02 -6.762915e+01\n'
b'  [7,] -1.712467e-06 -4.883569e-06 -3.041509e-06 -1.288544e-05  6.291198e-07\n'
b'  [8,] -9.743411e-07 -6.398075e-07 -2.059391e-06 -4.716032e-07 -4.314811e-07\n'
b'  [9,] -1.042754e+02 -4.396410e-07 -5.760538e-07 -1.140743e-06 -2.428050e+02\n'
b' [10,]  9.584099e-08 -1.901409e-07 -7.353104e-08 -7.346093e-08 -8.170041e+00\n'
b' [11,]  4.634529e-08 -1.84

b' [48,] -7.727357e-07 -1.616689e-06 -1.160730e+02 -1.669645e+02 -3.454093e-07\n'


In [54]:
poet = POET_wrapper(X)

b'[1] "Loaded X"\n'
b'[1] "Done with POET"\n'
b''


In [55]:
ls = LS_wrapper(X)

9657b8cb-c344-467c-893c-581128c6981c
b'[1] "Loaded X"\n'
b''


In [56]:
nls = NLS_wrapper(X)

b'[1] "Loaded X"\n'
b'Estimating population eigenvalues...[1] "Finished with NLS"\n'
b''


In [57]:
mtp2 = MTP2_wrapper(X)

1de06917-0ccc-49ca-9d30-13346fa21b3f


In [58]:
mtp2

array([[2.38309152e-04, 4.48899435e-05, 2.85415658e-05, ...,
        1.85453221e-05, 1.89633166e-05, 3.64375018e-05],
       [4.48899435e-05, 3.04107434e-04, 7.70784838e-05, ...,
        5.71164626e-05, 5.79407197e-05, 1.04381510e-04],
       [2.85415658e-05, 7.70784838e-05, 1.67835888e-04, ...,
        3.18517374e-05, 3.21783047e-05, 5.81361623e-05],
       ...,
       [1.85453221e-05, 5.71164626e-05, 3.18517374e-05, ...,
        7.47532379e-05, 2.33661058e-05, 4.27492572e-05],
       [1.89633166e-05, 5.79407197e-05, 3.21783047e-05, ...,
        2.33661058e-05, 1.06132737e-04, 4.13018188e-05],
       [3.64375018e-05, 1.04381510e-04, 5.81361623e-05, ...,
        4.27492572e-05, 4.13018188e-05, 2.38667882e-04]])

In [59]:
clime

array([[ 2.52447242e-04,  2.22458993e-08,  4.17940472e-09, ...,
         5.00451409e-10,  3.45125107e-09,  1.17452870e-07],
       [ 2.22458993e-08,  1.79418215e-04,  7.03504185e-07, ...,
         2.18933935e-08,  6.65180215e-09,  3.03187465e-06],
       [ 4.17940472e-09,  7.03504185e-07,  1.10531407e-04, ...,
        -1.29450975e-08,  1.71869962e-08,  9.47181489e-08],
       ...,
       [ 5.00451409e-10,  2.18933935e-08, -1.29450975e-08, ...,
         4.99826689e-05, -5.72978369e-10,  2.37638629e-08],
       [ 3.45125107e-09,  6.65180215e-09,  1.71869962e-08, ...,
        -5.72978369e-10,  8.46488529e-05,  1.10752104e-08],
       [ 1.17452870e-07,  3.03187465e-06,  9.47181489e-08, ...,
         2.37638629e-08,  1.10752104e-08,  1.80801497e-04]])

In [60]:
ls

array([[ 2.39371948e-04,  2.16806898e-05,  5.14245699e-06, ...,
        -3.58076522e-06,  7.00019880e-06,  1.80977025e-05],
       [ 2.16806898e-05,  2.98864085e-04,  4.71181358e-05, ...,
         4.55960709e-05,  4.71955386e-05,  8.07585830e-05],
       [ 5.14245699e-06,  4.71181358e-05,  1.75535232e-04, ...,
         2.73532205e-05,  1.50378405e-05,  4.18734840e-05],
       ...,
       [-3.58076522e-06,  4.55960709e-05,  2.73532205e-05, ...,
         9.12818343e-05,  2.11585442e-05,  3.04596251e-05],
       [ 7.00019880e-06,  4.71955386e-05,  1.50378405e-05, ...,
         2.11585442e-05,  1.19696611e-04,  2.62062174e-05],
       [ 1.80977025e-05,  8.07585830e-05,  4.18734840e-05, ...,
         3.04596251e-05,  2.62062174e-05,  2.39702403e-04]])

In [61]:
nls

array([[2.10005244e-04, 2.64861285e-05, 1.07654763e-05, ...,
        3.45447085e-06, 8.81504921e-06, 1.89477595e-05],
       [2.64861285e-05, 3.08311540e-04, 5.73170630e-05, ...,
        4.77568595e-05, 4.86977235e-05, 8.73348536e-05],
       [1.07654763e-05, 5.73170630e-05, 1.92946392e-04, ...,
        2.71575936e-05, 1.97853194e-05, 4.68244928e-05],
       ...,
       [3.45447085e-06, 4.77568595e-05, 2.71575936e-05, ...,
        1.34560944e-04, 2.06654611e-05, 3.24911120e-05],
       [8.81504921e-06, 4.86977235e-05, 1.97853194e-05, ...,
        2.06654611e-05, 1.51900625e-04, 2.99590074e-05],
       [1.89477595e-05, 8.73348536e-05, 4.68244928e-05, ...,
        3.24911120e-05, 2.99590074e-05, 2.39672102e-04]])

In [62]:
old_LS_wrapper(X)

array([[ 2.39373229e-04,  2.17044663e-05,  5.14809655e-06, ...,
        -3.58469213e-06,  7.00787568e-06,  1.81175496e-05],
       [ 2.17044663e-05,  2.98930609e-04,  4.71698086e-05, ...,
         4.56460746e-05,  4.72472963e-05,  8.08471482e-05],
       [ 5.14809655e-06,  4.71698086e-05,  1.75466506e-04, ...,
         2.73832178e-05,  1.50543320e-05,  4.19194052e-05],
       ...,
       [-3.58469213e-06,  4.56460746e-05,  2.73832178e-05, ...,
         9.11207104e-05,  2.11817480e-05,  3.04930291e-05],
       [ 7.00787568e-06,  4.72472963e-05,  1.50543320e-05, ...,
         2.11817480e-05,  1.19566648e-04,  2.62349569e-05],
       [ 1.81175496e-05,  8.08471482e-05,  4.19194052e-05, ...,
         3.04930291e-05,  2.62349569e-05,  2.39704048e-04]])